In [ ]:
import requests
import urllib.request
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from bs4 import BeautifulSoup
import pandas as pd
import json
from IPython.display import display, clear_output

In [ ]:
session = requests.Session()
retry = Retry(connect=10000, backoff_factor=5)
adapter = HTTPAdapter(max_retries=retry)

In [ ]:
talk_links = set()

In [ ]:
next_page = "https://www.ted.com/talks"

while True:
    
    response = session.get(next_page)

    soup = BeautifulSoup(response.text, "html.parser")

    link_as = soup.findAll('a')
    
    next_page = None
    
    for link_a in link_as:
        if 'data-ga-context="talks"' in str(link_a):
            talk_links.add("https://www.ted.com"+link_a["href"]+"/transcript?language=en")
        if 'rel="next"' in str(link_a):
            next_page = "https://www.ted.com"+link_a["href"]
    
    if next_page is None:
        break

In [ ]:
len_talklinks = len(talk_links)

In [ ]:
data = []

i = 1
failed = 0

for talk_link in talk_links:

    response = session.get(talk_link)
    
    soup = BeautifulSoup(response.text, "html.parser")
    
    if "Sorry, we can’t find that." in str(soup):
        failed += 1
        pass
        
    else:
        page_info = [i.string.lstrip('q("talkPage.init", ').rstrip(')') for i in soup.select('script') if i.string and i.string.startswith('q')][0]

        page_info_data = json.loads(page_info)["__INITIAL_DATA__"]

        tags = (page_info_data["talks"][0]["tags"])
        
        link_ps = soup.findAll('p')

        script = ""

        for link_p in link_ps:
            if "TED.com translations are made possible by volunteer" in link_p.text:
                break
            script += (' '.join(str(link_p.text).split())) + "\n"
            
        for tag in tags:
            data.append([tag, script])
            
    clear_output(wait=True)
    display('Percentage: '+str(i*100/len_talklinks) + '%, Failed: '+str(failed*100/len_talklinks)+"%")
    i+=1

In [ ]:
df = pd.DataFrame (data, columns = ["Tag", "Script"])

In [ ]:
df.to_csv('script.csv', encoding='utf-8', index=False)